In [1]:
# Carregando algumas bibliotecas importantes
import numpy as np
import pandas as pd
from datetime import datetime

In [2]:
# Importando os arquivos
jogos = pd.read_csv('../data/raw/matches_FLA2024.csv', date_format={'kick-off': '%Y-%m-%d %H:%M:%S'})
gols = pd.read_csv('../data/raw/goal_scorers_2024.csv')

In [3]:
jogos['kick-off'] = pd.to_datetime(jogos['kick-off'], errors='coerce')
jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73 entries, 0 to 72
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id_match    73 non-null     object        
 1   home_team   73 non-null     object        
 2   away_team   73 non-null     object        
 3   home_score  73 non-null     int64         
 4   away_score  73 non-null     int64         
 5   tournament  73 non-null     object        
 6   kick-off    73 non-null     datetime64[ns]
 7   stadium     73 non-null     object        
 8   attendance  73 non-null     int64         
dtypes: datetime64[ns](1), int64(3), object(5)
memory usage: 5.3+ KB


In [4]:
# Mostrando as 5 primeiras linhas do dataframe jogos
jogos.head()

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance
0,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068
1,AM1,Philadelphia Union-EUA,Flamengo,0,2,International Friendly Game,2024-01-21 16:00:00,AI Lang Stadium-EUA,7227
2,CA2,Nova Iguaçu-RJ,Flamengo,1,1,Carioca - Round 2,2024-01-21 18:10:00,Estádio Almeidão-PB,16428
3,AM2,Orlando City,Flamengo,1,1,International Friendly Game,2024-01-27 16:00:00,Inter&Co Stadium-EUA,22678
4,CA3,Portuguesa-RJ,Flamengo,0,0,Carioca - Round 3,2024-01-27 18:10:00,Arena das Dunas-RN,25211


In [5]:
# Mostrando as 5 ultimas linhas do dataframe jogos
jogos.tail()

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance
68,CB10,Atlético-MG,Flamengo,0,1,Copa do Brasil - Final - Second Leg,2024-11-10 16:00:00,Arena MRV-MG,44876
69,BR33,Flamengo,Atlético-MG,0,0,Campeonato Brasileiro - Round 33,2024-11-13 20:00:00,Maracanã-RJ,63441
70,BR34,Cuiabá,Flamengo,1,2,Campeonato Brasileiro - Round 34,2024-11-20 19:00:00,Arena Pantanal-MT,12891
71,BR35,Fortaleza,Flamengo,0,0,Campeonato Brasileiro - Round 35,2024-11-26 20:00:00,Arena Castelão-CE,56646
72,BR36,Flamengo,Internacional,3,2,Campeonato Brasileiro - Round 36,2024-12-01 16:00:00,Maracanã-RJ,56571


In [6]:
# Mostrando as 5 primeiras linhas do dataframe jogos
gols.head()

,id_match,gols,assist
0,CA1,Léo Pereira,Arrascaeta
1,CA1,Pedro,Gerson
2,CA1,Everton Cebolinha,NaN
3,CA1,Varela,Luiz Araújo
4,AM1,Pedro,NaN


In [7]:
# Mostrando as 5 ultimas linhas do dataframe jogos
gols.tail()

,id_match,gols,assist
107,BR34,Guilherme,Bruno Henrique
108,BR34,Matheus Gonçalves,Michael
109,BR36,Léo Ortiz,de la Cruz
110,BR36,Michael,Alex Sandro
111,BR36,Michael,NaN


### Criando a coluna torneio

In [9]:
# Split o nome do torneio e a rodada
lista = [i.split(' -', 1)[0] for i in jogos["tournament"].tolist()]

# Transformando em uma nova coluna chamada torneio
jogos['torneio'] = np.array(lista)

### Criando a coluna local (casa/fora)

In [11]:
def obter_locale(row):
    if row['torneio'] == 'Supercopa do Brasil' or row['torneio'] == 'Club World Championship':
        return 'Neutro'
    elif row['home_team'] == 'Flamengo':
        return 'Casa'
    else:
        return 'Fora'

    
# Aplicar a função para criar a nova coluna
jogos['locale'] = jogos.apply(obter_locale, axis=1)

### Criando a coluna resultado

In [13]:
# Função para obter resultado
def obter_resultado(row):
    if row['home_team'] == 'Flamengo':
        gols_marcados = row['home_score']
        gols_sofridos = row['away_score']
        
        if gols_marcados < gols_sofridos:
            return 'Derrota'
        elif gols_marcados > gols_sofridos:
            return 'Vitória'
        else:
            return 'Empate'
    if row['away_team'] == 'Flamengo':
        gols_marcados = row['away_score']
        gols_sofridos = row['home_score']

        if gols_marcados < gols_sofridos:
            return 'Derrota'
        elif gols_marcados > gols_sofridos:
            return 'Vitória'
        else:
            return 'Empate'

# Aplicar a função para criar a nova coluna
jogos['resultado'] = jogos.apply(obter_resultado, axis=1)

### Criando mais algumas colunas 

In [15]:
def gerar_estatisticas_gols(row):
    if row['home_team'] == 'Flamengo':
        gols_feitos_casa = row['home_score']
        gols_sofridos_casa = row['away_score']
        gols_feitos_fora = 0
        gols_sofridos_fora = 0
        marcou_gol = 'Sim' if row['home_score'] > 0 else 'Não'
        sofreu_gol = 'Sim' if row['away_score'] > 0 else 'Não'
    elif row['away_team'] == 'Flamengo':
        gols_feitos_casa = 0
        gols_sofridos_casa = 0
        gols_feitos_fora = row['away_score']
        gols_sofridos_fora = row['home_score']
        marcou_gol = 'Sim' if row['away_score'] > 0 else 'Não'
        sofreu_gol = 'Sim' if row['home_score'] > 0 else 'Não'
    else:
        gols_feitos_casa = 0
        gols_sofridos_casa = 0
        gols_feitos_fora = 0
        gols_sofridos_fora = 0
        marcou_gol = 'Não'
        sofreu_gol = 'Não'

    return pd.Series([gols_feitos_casa, gols_sofridos_casa, gols_feitos_fora, gols_sofridos_fora, marcou_gol, sofreu_gol])

jogos[['gols_casa', 'gols_sofridos_casa', 'gols_fora', 'gols_sofridos_fora', 'marcou_gol', 'sofreu_gol']] = jogos.apply(gerar_estatisticas_gols, axis=1)        

jogos['jogos_cumulativos'] = range(1, len(jogos) + 1)

#### Salvando os dataframes 

In [17]:
jogos['year'] = jogos['kick-off'].map(lambda x: datetime.strftime(x,"%Y"))
jogos['month'] = jogos['kick-off'].map(lambda x: datetime.strftime(x,"%b"))
jogos['weekday'] = jogos['kick-off'].map(lambda x: datetime.strftime(x,"%a"))

In [18]:
jogos.to_csv('../data/processed/matches_FLA2024_processed.csv', index=False)
gols.to_csv('../data/processed/goal_scorers_2024_processed.csv', index=False)

In [19]:
jogos

,id_match,home_team,away_team,home_score,away_score,tournament,kick-off,stadium,attendance,torneio,...,gols_casa,gols_sofridos_casa,gols_fora,gols_sofridos_fora,marcou_gol,sofreu_gol,jogos_cumulativos,year,month,weekday
0,CA1,Flamengo,Audax-RJ,4,0,Carioca - Round 1,2024-01-17 21:30:00,Arena da Amazônia-AM,44068,Carioca,...,4,0,0,0,Sim,Não,1,2024,Jan,Wed
1,AM1,Philadelphia Union-EUA,Flamengo,0,2,International Friendly Game,2024-01-21 16:00:00,AI Lang Stadium-EUA,7227,International Friendly Game,...,0,0,2,0,Sim,Não,2,2024,Jan,Sun
2,CA2,Nova Iguaçu-RJ,Flamengo,1,1,Carioca - Round 2,2024-01-21 18:10:00,Estádio Almeidão-PB,16428,Carioca,...,0,0,1,1,Sim,Sim,3,2024,Jan,Sun
3,AM2,Orlando City,Flamengo,1,1,International Friendly Game,2024-01-27 16:00:00,Inter&Co Stadium-EUA,22678,International Friendly Game,...,0,0,1,1,Sim,Sim,4,2024,Jan,Sat
4,CA3,Portuguesa-RJ,Flamengo,0,0,Carioca - Round 3,2024-01-27 18:10:00,Arena das Dunas-RN,25211,Carioca,...,0,0,0,0,Não,Não,5,2024,Jan,Sat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68,CB10,Atlético-MG,Flamengo,0,1,Copa do Brasil - Final - Second Leg,2024-11-10 16:00:00,Arena MRV-MG,44876,Copa do Brasil,...,0,0,1,0,Sim,Não,69,2024,Nov,Sun
69,BR33,Flamengo,Atlético-MG,0,0,Campeonato Brasileiro - Round 33,2024-11-13 20:00:00,Maracanã-RJ,63441,Campeonato Brasileiro,...,0,0,0,0,Não,Não,70,2024,Nov,Wed
70,BR34,Cuiabá,Flamengo,1,2,Campeonato Brasileiro - Round 34,2024-11-20 19:00:00,Arena Pantanal-MT,12891,Campeonato Brasileiro,...,0,0,2,1,Sim,Sim,71,2024,Nov,Wed
71,BR35,Fortaleza,Flamengo,0,0,Campeonato Brasileiro - Round 35,2024-11-26 20:00:00,Arena Castelão-CE,56646,Campeonato Brasileiro,...,0,0,0,0,Não,Não,72,2024,Nov,Tue
